In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
#!rm -rf /content/vanGogh-and-Other-Artist
!git clone https://github.com/czkaiweb/vanGogh-and-Other-Artist.git

# To fetch the change from git repo
%cd /content/vanGogh-and-Other-Artist
!git fetch 
!git pull
!git checkout develop
%cd /content

Cloning into 'vanGogh-and-Other-Artist'...
remote: Enumerating objects: 169, done.
remote: Counting objects: 100% (169/169), done.
remote: Compressing objects: 100% (114/114), done.
remote: Total 169 (delta 99), reused 115 (delta 51), pack-reused 0
Receiving objects: 100% (169/169), 654.35 KiB | 3.94 MiB/s, done.
Resolving deltas: 100% (99/99), done.
/content/vanGogh-and-Other-Artist
Already up to date.
Branch 'develop' set up to track remote branch 'develop' from 'origin'.
Switched to a new branch 'develop'
/content


In [18]:
import sys
sys.path.append('/content/vanGogh-and-Other-Artist')
sys.path.append('/content/vanGogh-and-Other-Artist/preprocessing')
sys.path.append('/content/vanGogh-and-Other-Artist/model')
sys.path.append('/content/ModelWeights')
from genericVoter import *
from preprocessing.ImageTranform import *
from torchsummary import summary

import shutil
import os
import glob
import pandas as pd
import numpy
from tqdm import tqdm
import hashlib

from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

In [19]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
# Create Meta record
fileList = []
img_path = "./imgs"

if not os.path.isdir(img_path):
    os.mkdir(img_path)

In [21]:
kaggle_path = "/root/.kaggle"
if not os.path.isdir(kaggle_path):
  os.mkdir(kaggle_path)

In [22]:
!chmod 600 /root/.kaggle/kaggle.json

In [8]:
# Download van Gogh dataset
!kaggle datasets download -d ipythonx/van-gogh-paintings

# List all von Goph plots
tmp_path = "./tmp"
vangoghZip = 'van-gogh-paintings.zip'
try:
    shutil.unpack_archive(vangoghZip,tmp_path)
except Exception as err:
    print(err)

allVanGogh = glob.glob(tmp_path+'/*/*.jpg')

# Append metadata
for index in tqdm(range(len(allVanGogh))):
    fileName = allVanGogh[index]
    file = fileName.split("/")[-1]
    hashName = hashlib.md5(file.encode()).hexdigest()
    shutil.move(fileName, img_path + "/" + hashName + ".jpg", copy_function = shutil.copy2)
    artist = "vanGogh"
    fileList.append([hashName,artist])
    
# Clean tmp data
try:
    shutil.rmtree(tmp_path)
    os.remove(vangoghZip)
except Exception as err:
    print(err)

 95% 462M/485M [00:03<00:00, 189MB/s]
100% 485M/485M [00:03<00:00, 161MB/s]


100%|██████████| 2024/2024 [00:00<00:00, 10145.87it/s]


In [9]:
!kaggle datasets download -d srrrrr/monet2photo

# List all Monet plots
tmp_path = "./tmp"
monetZip = 'monet2photo.zip'
try:
    shutil.unpack_archive(monetZip,tmp_path)
except Exception as err:
    print(err)

allMonet = glob.glob(tmp_path+'/*/trainA/*.jpg')

# Append metadata
for index in tqdm(range(len(allMonet))):
    fileName = allMonet[index]
    file = fileName.split("/")[-1]
    hashName = hashlib.md5(file.encode()).hexdigest()
    shutil.move(fileName, img_path + "/" + hashName + ".jpg", copy_function = shutil.copy2)
    artist = "Monet"
    fileList.append([hashName,artist])
    
# Clean tmp data
try:
    shutil.rmtree(tmp_path)
    os.remove(monetZip)
except Exception as err:
    print(err)

 99% 259M/263M [00:02<00:00, 111MB/s]
100% 263M/263M [00:02<00:00, 102MB/s]


100%|██████████| 1072/1072 [00:00<00:00, 22301.28it/s]


In [10]:
# Download WikiArts dataset: https://www.kaggle.com/datasets/antoinegruson/-wikiart-all-images-120k-link
!kaggle datasets download -d czkaiweb/subwikiarts

# List all wikiarts plots
tmp_path = "./tmp"
wikiartsZip = 'subwikiarts.zip'
try:
    shutil.unpack_archive(wikiartsZip,tmp_path)
except Exception as err:
    print(err)

WikiArtsMeta = tmp_path+"/WikiArts.csv"
WikiArtsDF = pd.read_csv(WikiArtsMeta)
WikiArtsList = WikiArtsDF[["hash","Artist"]].values

def findGroup(head):
    if head <= "33":
        return "/GroupA/"
    elif head <= "69":
        return "/GroupB/"
    elif head <= "9d":
        return "/GroupC/"
    elif head <= "cc":
        return "/GroupD/"
    else:
        return "/GroupE/"
    
# Set to true for group splitting
preClean = False
if preClean == True:
    for char in ["A","B","C","D","E"]:
        groupDir = img_path+"/Group{}".format(char)
        if not os.path.isdir(groupDir):
            os.mkdir(groupDir)

for record in WikiArtsList:
    groupDir = "/./"
    if preClean:
        groupDir = findGroup(record[0][:2])
    fileName = tmp_path+"/imgs/"+record[0]+".jpg"
    shutil.move(fileName, img_path+ groupDir + "/" , copy_function = shutil.copy2)
    
# Clean tmp data
try:
    shutil.rmtree(tmp_path)
    os.remove(wikiartsZip)
except Exception as err:
    print(err)


 99% 989M/995M [00:10<00:00, 103MB/s]
100% 995M/995M [00:10<00:00, 98.2MB/s]


In [11]:
# Save to meta file
metaDF = pd.DataFrame(fileList,columns = ["hash","Artist"])
metaDF = pd.concat([metaDF,WikiArtsDF[["hash","Artist"]]])
metaDF.to_csv("meta.csv")

In [23]:
myVoter = genericVoter()

In [42]:
listTransformer = []
listModel = []
listWeight = []

# For vgg16
myTransform = ImageTransformer((224,224))
myTransform.initTransform()
transformer = myTransform.getTransformer()
listTransformer.append(transformer)

model_vgg = models.vgg16()
num_ftrs = model_vgg.classifier[6].in_features
model_vgg.classifier[6] = nn.Linear(num_ftrs, 6)
model_vgg = model_vgg.to(myVoter.device)
#model_vgg.load_state_dict(torch.load('/content/drive/MyDrive/VGG16_weights_May28.pth'))
model_vgg.load_state_dict(torch.load('/content/ModelWeights/VGG16_weights_May28.pth'))
listModel.append(model_vgg)

listWeight.append('/content/ModelWeights/VGG16_weights_May28.pth')

# For EfficientNet
myTransform = ImageTransformer((224,224))
myTransform.initTransform()
transformer = myTransform.getTransformer()
listTransformer.append(transformer)

model_effnet = models.efficientnet_b0()
num_ftrs = model_effnet.classifier[1].in_features
model_effnet.classifier[1] = nn.Linear(num_ftrs, 6)
model_effnet = model_effnet.to(myVoter.device)
model_effnet.load_state_dict(torch.load('/content/ModelWeights/model_weights_EfficientNetB0_unnormalized_final.pth'))
listModel.append(model_effnet)

listWeight.append('/content/ModelWeights/model_weights_EfficientNetB0_unnormalized_final.pth')

# For MobileNet_v2
myTransform = ImageTransformer((224,224))
myTransform.initTransform()
transformer = myTransform.getTransformer()
listTransformer.append(transformer)

model_mobnet = models.mobilenet_v2()
num_ftrs = model_mobnet.classifier[1].in_features
model_mobnet.classifier[1] = nn.Linear(num_ftrs, 6)
model_mobnet = model_mobnet.to(myVoter.device)
model_mobnet.load_state_dict(torch.load('/content/ModelWeights/model_weights_mobilenet_v2_valp1trainp2.pth'))
listModel.append(model_mobnet)

listWeight.append('/content/ModelWeights/model_weights_mobilenet_v2_valp1trainp2.pth')

# For ResNet
myTransform = ImageTransformer((224,224))
myTransform.initTransform()
transformer = myTransform.getTransformer()
listTransformer.append(transformer)

model_resnet = models.resnet34()
num_ftrs = model_resnet.fc.in_features
model_resnet.fc = nn.Linear(num_ftrs, 6)
model_resnet = model_resnet.to(myVoter.device)
model_resnet.load_state_dict(torch.load('/content/ModelWeights/model_weights_resnet34.pth'))
listModel.append(model_resnet)

listWeight.append('/content/ModelWeights/model_weights_resnet34.pth')

# For ConvNext
myTransform = ImageTransformer((224,224))
myTransform.initTransform()
transformer = myTransform.getTransformer()
listTransformer.append(transformer)

model_convnext = models.convnext_base()
num_ftrs = model_convnext.classifier[2].in_features
model_convnext.classifier[2] = nn.Linear(num_ftrs, 6)
model_convnext = model_convnext.to(myVoter.device)
model_convnext.load_state_dict(torch.load('/content/ModelWeights/model_weights_convNext.pth'))
listModel.append(model_convnext)

listWeight.append('/content/ModelWeights/model_weights_convNext.pth')

myVoter.setBagging(listModel,listTransformer,listWeight)

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:1421: UserWarning: The parameter 'resample' is deprecated since 0.12 and will be removed in 0.14. Please use 'interpolation' instead.
  "The parameter 'resample' is deprecated since 0.12 and will be removed in 0.14. "
/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:1436: UserWarning: The parameter 'fillcolor' is deprecated since 0.12 and will be removed in 0.14. Please use 'fill' instead.
  "The parameter 'fillcolor' is deprecated since 0.12 and will be removed in 0.14. "


In [43]:
myTransform = ImageTransformer((224,224))
myTransform.initTransform()
transformer = myTransform.getTransformer()
myVoter.setTransformer(transformer)

# Set up the meta data and path to image dataset
myVoter.setDataset("meta.csv",path = "imgs")

# Split the data by portion, fraction indicate the percentage of data used in the whole dataset. 
# Default: val_size = 0.2, test_size = 0.1 
myVoter.splitData(val_size=0.1,test_size = 0.7,fraction = 1)

# Will automatically get the statistic for training set, update the mean/std used for normalization. 
# loadData and checkDataset
myVoter.loadData()
myVoter.checkDataset()
myVoter.trainMean = torch.tensor([0.5396, 0.4863, 0.4146])
myVoter.trainStd = torch.tensor([0.2776, 0.2715, 0.2588])

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:1421: UserWarning: The parameter 'resample' is deprecated since 0.12 and will be removed in 0.14. Please use 'interpolation' instead.
  "The parameter 'resample' is deprecated since 0.12 and will be removed in 0.14. "
/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:1436: UserWarning: The parameter 'fillcolor' is deprecated since 0.12 and will be removed in 0.14. Please use 'fill' instead.
  "The parameter 'fillcolor' is deprecated since 0.12 and will be removed in 0.14. "


53e872e09c40a912e36d53daf6920243 torch.Size([4, 224, 224])
0641ceb25ff823cf52802ea8c07558b1 torch.Size([1, 224, 224])
ee7c58e67dabd4b7459bddf9f669f707 torch.Size([1, 224, 224])
84e717a6cf749d681ffbb2557758bd13 torch.Size([1, 224, 224])
7dc6002a966d35a49f263376c2a56256 torch.Size([1, 224, 224])
8c9f5633d217a54f78ed5f2146324b38 torch.Size([1, 224, 224])
632fc73d4e2de07f28d85429ff390476 torch.Size([1, 224, 224])
b96a4a83299a54b4be4ff78c0d92c170 torch.Size([1, 224, 224])
a2bbef5813e42924f2168bef12a26478 torch.Size([4, 224, 224])
9d694f7ee70d5f694dda849a0a132bcb torch.Size([4, 224, 224])
d7999f4ec0a7efcc004c733e813ddb39 torch.Size([4, 224, 224])
0be2cd049d2ecf361c6ffcb1c5054de5 torch.Size([4, 224, 224])
e2ee5d0fd45f95074b7e5e8665ab22de torch.Size([4, 224, 224])
bcbf898f298ba2a00e088c50e3e27134 torch.Size([1, 224, 224])
e5f0411ee34ba439116a0d8d8cfe64c3 torch.Size([1, 224, 224])
042faaee15402a071df98949a2fe4298 torch.Size([1, 224, 224])
521cde1dc6bf903b92698cce5fc45cc9 torch.Size([1, 224, 224

In [44]:
myVoter.prepareInputForVoter(skipNorm = True)

100%|██████████| 565/565 [00:56<00:00, 10.07it/s]

Sanity check for model:0, accuracy:0.8707964601769912
Sanity check for model:1, accuracy:0.9061946902654867
Sanity check for model:2, accuracy:0.8548672566371681
Sanity check for model:3, accuracy:0.8407079646017699
Sanity check for model:4, accuracy:0.8920353982300885


In [45]:
import xgboost as xgb
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

voters = {
#    "xgboost"  : xgb.XGBClassifier(objective="multi:softprob", random_state=42),
    "kNeighbor" : KNeighborsClassifier(3),
    "svm_rbf"  : svm.SVC(kernel='rbf', gamma=0.5, C=0.1),
    "svm_poly" : svm.SVC(kernel='poly', degree=3, C=1),
    "randomForest": RandomForestClassifier(n_estimators=5, random_state=1),
    "decisionTree" :  DecisionTreeClassifier(max_depth=3),
    #"Logistic": LogisticRegression(multi_class='multinomial',random_state=1)
}

myVoter.highaccuracy = None
myVoter.bestVoter = None
myVoter.setVoterClassifier(voters)
myVoter.fitVoter(useKFold=False)


kNeighbor accuracy: 0.9274336283185841
svm_rbf accuracy: 0.8477876106194691
svm_poly accuracy: 0.727433628318584
randomForest accuracy: 0.9610619469026549
decisionTree accuracy: 0.8035398230088495


In [46]:
#myVoter.bestVoter.predict([[1,1,3]])
myVoter.evaluateVoter()

100%|██████████| 3952/3952 [11:42<00:00,  5.63it/s]


voter accuracy: 0.8805668016194332
base model: 0 accu: 0.8307186234817814
base model: 1 accu: 0.8833502024291497
base model: 2 accu: 0.8656376518218624
base model: 3 accu: 0.8512145748987854
base model: 4 accu: 0.867661943319838
hard voting accu: 0.9018218623481782
